In [2]:
import simpy
import numpy as np
from evaluation.plot import evolution
from elements.workflow_process_elements import StartEvent, UserTask, connect
from evaluation.statistics import calculate_statistics
from policies.monte_carlo import MC
from simulations import *
%matplotlib notebook


# init q_table and reinforcement learning variables
q_table = np.zeros((100, 100, NUMBER_OF_USERS))
epsilon = 0.1
gamma = 0.9
epochs = 3

for i in range(epochs):
    # creates simulation environment
    env = simpy.Environment()

    # open file and write header
    file_policy,file_statistics,file_policy_name,file_statistics_name = create_files("run{}_mc".format(i))

    # initialize policy
    policy = MC(env, NUMBER_OF_USERS, WORKER_VARAIBILITY, file_policy, file_statistics,q_table,epsilon,gamma)

    # start event
    start_event = StartEvent(env, GENERATION_INTERVAL)

    # user tasks
    user_task = UserTask(env, policy, "User task 1", SERVICE_INTERVAL, TASK_VARIABILITY)

    # connections
    connect(start_event, user_task)

    # calls generation tokens process
    env.process(start_event.generate_tokens())

    # runs simulation
    env.run(until=SIM_TIME)

    # update q_table
    new_q_table = MC.update_q_table(policy)
    q_table = new_q_table

    # close file
    file_policy.close()
    file_statistics.close()

    # calculate statistics and plots
    # calculate_statistics(file_policy_name, outfile="{}.pdf".format(file_policy_name[:-4]))
    evolution(file_statistics_name)

<IPython.core.display.Javascript object>

<div id='00cee80b-600f-4622-891b-0ea50334e5fe'></div>

<IPython.core.display.Javascript object>

<div id='e3e0efc2-24b0-40ad-ba96-05c2e1d3c7ab'></div>

<IPython.core.display.Javascript object>

...

<div id='7e9631a8-7273-459f-a01a-2cde5c095d0f'></div>

<IPython.core.display.Javascript object>

<div id='3834fd65-f894-48de-871b-2258f7d5960d'></div>

<IPython.core.display.Javascript object>

<div id='afe9f92e-5063-44d5-bd68-6eb033e6d409'></div>